<a href="https://colab.research.google.com/github/tasdid25/Django/blob/main/django_crud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Django-তে CRUD অপারেশন**


## **Step 1: Model তৈরি করা**
প্রথমে **models.py** ফাইলে একটি **Book** মডেল তৈরি করতে হবে।

In [ ]:
from django.db import models

class Book(models.Model):
    title = models.CharField(max_length=255)
    author = models.CharField(max_length=255)
    published_date = models.DateField()

    def __str__(self):
        return self.title

## **Step 2: Database মাইগ্রেশন করা**
মডেল তৈরি করার পরে **মাইগ্রেশন** চালিয়ে তা ডাটাবেজে যোগ করতে হবে।

In [ ]:
!python manage.py makemigrations
!python manage.py migrate

## **Step 3: Views সেটআপ করা**
Django-তে CRUD অপারেশনের জন্য **views.py** ফাইলে নিচের মতো কোড লিখতে হবে।
     

*   1. fetch all data of book form database
*   2. send all fetched data to a template





### **(C) Create - নতুন বই যোগ করা**

In [ ]:
from django.shortcuts import render, redirect
from .models import Book
from .forms import BookForm

# নতুন বই create করার ফাংশন
# Handles both GET (ফর্ম দেখানো) এবং POST (ফর্ম সাবমিট করা) রিকোয়েস্ট
def create_book(request):
    if request.method == "POST":  # 1. get the all data from frontend
        form = BookForm(request.POST)
        if form.is_valid():  # ফর্ম ডাটা যাচাই করা
            form.save()  # নতুন বই ডাটাবেসে save করা
            return redirect('book_list')  # সফলভাবে save হলে বই তালিকায় নিয়ে যাওয়া
    else:
        form = BookForm()  # GET রিকোয়েস্টের জন্য ফাঁকা ফর্ম দেখানো
    return render(request, 'books/create_book.html', {'form': form})  # ফর্ম সহ HTML টেমপ্লেট রেন্ডার করা


### **(R) Read - সব বই লিস্ট আকারে দেখানো**

*   fetch all data of book form database

*   send all fetched data to a template



In [ ]:
def book_list(request):
    books = Book.objects.all()  # ডাটাবেস থেকে সমস্ত বই নিয়ে আসা
    return render(request, 'books/book_list.html', {'books': books})  # বই তালিকা টেমপ্লেটে পাঠানো

### **(U) Update - বিদ্যমান বই আপডেট করা**

In [ ]:
def update_book(request, pk):
    book = Book.objects.get(id=pk)  # নির্দিষ্ট বই খুঁজে বের করা
    if request.method == "POST":  # যদি ফর্ম সাবমিট করা হয়
        form = BookForm(request.POST, instance=book)  # বিদ্যমান বইয়ের তথ্যসহ ফর্ম তৈরি করা
        if form.is_valid():  # ফর্ম ডাটা যাচাই করা
            form.save()  # পরিবর্তিত তথ্য ডাটাবেসে save করা
            return redirect('book_list')  # আপডেট সফল হলে বই তালিকায় নিয়ে যাওয়া
    else:
        form = BookForm(instance=book)  # GET রিকোয়েস্টের জন্য ফর্মে বিদ্যমান তথ্য দেখানো
    return render(request, 'books/update_book.html', {'form': form})  # ফর্ম সহ HTML টেমপ্লেট রেন্ডার করা

### **(D) Delete - বই ডিলিট করা**

In [ ]:
def delete_book(request, pk):
    book = Book.objects.get(id=pk)  # নির্দিষ্ট বই খুঁজে বের করা
    if request.method == "POST":  # যদি delete কনফার্ম করা হয়
        book.delete()  # বই ডাটাবেস থেকে মুছে ফেলা
        return redirect('book_list')  # সফলভাবে delete হলে বই তালিকায় নিয়ে যাওয়া
    return render(request, 'books/delete_book.html', {'book': book})  # কনফার্মেশন টেমপ্লেট রেন্ডার করা

## **Step 4: URL সেটআপ করা**

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('', views.book_list, name='book_list'),
    path('create/', views.create_book, name='create_book'),
    path('update/<int:pk>/', views.update_book, name='update_book'),
    path('delete/<int:pk>/', views.delete_book, name='delete_book'),
]

## **Step 5: ফর্ম তৈরি করা**

In [ ]:
from django import forms
from .models import Book

class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author', 'published_date']

## **Step 6: Superuser তৈরি করা (Admin Panel)**

In [ ]:
!python manage.py createsuperuser

In [ ]:
from django.contrib import admin
from .models import Book

admin.site.register(Book)